# Domain_walls

### domain_wall.py

In [1]:
from ase import Atoms
from ase.build import cut, rotate, stack
from ase.geometry import find_mic
from ase.io import read
from ase.neighborlist import NeighborList
from ase.spacegroup import get_spacegroup
from ase.visualize import view
from functools import reduce
from itertools import product
from math import gcd
from numpy.linalg import norm
from pyqapi.core import *
from pyqapi.core.mp_data import get_cif_by_id

import io
import numpy as np

class DomainWallSystem:
    def __init__(self, atoms, **kwargs):
        self.atoms = atoms
        self.international_symbol, _ = self.identify_system_symmetry()
        self.domain_wall_tag = kwargs.get('domain_wall_tag', "180")

        if self.domain_wall_tag == "customed":
            required_params = ['domainA_a', 'domainA_b', 'domainA_c', 'domainB_a', 'domainB_b', 'domainB_c',
                               'stack_axis']
            for param in required_params:
                value = kwargs.get(param)
                if value is None:
                    raise ValueError(f"Parameter '{param}' must be provided.")

            self.domainA_a = kwargs.get('domainA_a')
            self.domainA_b = kwargs.get('domainA_b')
            self.domainA_c = kwargs.get('domainA_c')
            self.domainB_a = kwargs.get('domainB_a')
            self.domainB_b = kwargs.get('domainB_b')
            self.domainB_c = kwargs.get('domainB_c')
            self.stack_axis = kwargs.get('stack_axis')  # 0/1/2
        else:
            self.domain_size = kwargs.get('domain_size', 3)
            predefined_systems = self.predefined_systems()
            if self.international_symbol not in predefined_systems or str(self.domain_wall_tag) not in \
                    predefined_systems[self.international_symbol]:
                raise ValueError(
                    f"The combination of international symbol '{self.international_symbol}' and domain wall tag '{self.domain_wall_tag}' is not defined in predefined systems.")

            predefined_values = predefined_systems[self.international_symbol][str(self.domain_wall_tag)]
            self.domainA_a, self.domainA_b, self.domainA_c, self.stack_axis = predefined_values[0].values()
            self.domainB_a, self.domainB_b, self.domainB_c, _ = predefined_values[1].values()

        self.cutoff = kwargs.get('cutoff', 0.0)

    def identify_system_symmetry(self):
        symmetry = get_spacegroup(self.atoms, symprec=1e-5)
        international_symbol = symmetry.symbol
        spacegroup_number = symmetry.no
        return international_symbol, spacegroup_number

    def predefined_systems(self):
        predefined_domain_wall_dict = {
            "R 3 m": {
                "71": [
                    {'a': [self.domain_size, self.domain_size, 0], 'b': [0, 0, 1], 'c': [1, -1, 0], 'stack_axis': 0},
                    {'a': [self.domain_size, self.domain_size, 0], 'b': [0, 0, -1], 'c': [-1, 1, 0], 'stack_axis': 0}
                ],
                "109": [
                    {'a': [self.domain_size, 0, 0], 'b': [0, 1, 0], 'c': [0, 0, 1], 'stack_axis': 0},
                    {'a': [self.domain_size, 0, 0], 'b': [0, -1, 0], 'c': [0, 0, -1], 'stack_axis': 0}
                ],
                "180": [
                    {'a': [1, 1, 0], 'b': [0, 0, 1], 'c': [self.domain_size, -self.domain_size, 0], 'stack_axis': 2},
                    {'a': [-1, -1, 0], 'b': [0, 0, -1], 'c': [self.domain_size, -self.domain_size, 0], 'stack_axis': 2}
                ]
            },
            "R 3 c": {
                "71": [
                    {'a': [1, -1, 0], 'b': [0, 0, 1], 'c': [self.domain_size, self.domain_size, 0], 'stack_axis': 2},
                    {'a': [-1, 1, 0], 'b': [0, 0, -1], 'c': [self.domain_size, self.domain_size, 0], 'stack_axis': 2}
                ],
                "109": [
                    {'a': [0.0, 1, 0], 'b': [1, 0, 0], 'c': [0, 0, self.domain_size], 'stack_axis': 2},
                    {'a': [0.0, -1, 0], 'b': [-1, 0, 0], 'c': [0, 0, self.domain_size], 'stack_axis': 2}
                ],
                "180": [
                    {'a': [1, 1, 0], 'b': [0, 0, -1], 'c': [-self.domain_size, self.domain_size, 0], 'stack_axis': 2},
                    {'a': [-1, -1, 0], 'b': [0, 0, 1], 'c': [-self.domain_size, self.domain_size, 0], 'stack_axis': 2}
                ]
            },
            "P 4 m m": {
                "90": [
                    {'a': [0, 1, 0], 'b': [-1, 0, 1], 'c': [self.domain_size, 0, self.domain_size], 'stack_axis': 2},
                    {'a': [0, -1, 0], 'b': [1, 0, -1], 'c': [self.domain_size, 0, self.domain_size], 'stack_axis': 2}
                ],
                "180": [
                    {'a': [1.01, 0, 0], 'b': [0, self.domain_size, 0], 'c': [0, 0, 1.01], 'stack_axis': 1},
                    {'a': [-1.01, 0, 0], 'b': [0, self.domain_size, 0], 'c': [0, 0, -1.01], 'stack_axis': 1}
                ]
            },
            "P m c 21": {
                "90": [
                    {'a': [0, 1 * self.domain_size, 0], 'b': [-1, 0, 0], 'c': [0, 0, 1], 'stack_axis': 0},
                    {'a': [-1 * self.domain_size, 0, 0], 'b': [0, -1, 0], 'c': [0, 0, 1], 'stack_axis': 0}
                ],
                "120_HH_TT": [
                    {'a': [-1, -2, 0], 'b': [2.0 * self.domain_size, -1.0 * self.domain_size, 0], 'c': [0, 0, 1],
                     'stack_axis': 1},
                    {'a': [-1, 2, 0], 'b': [-2.0 * self.domain_size, -1.0 * self.domain_size, 0], 'c': [0, 0, 1],
                     'stack_axis': 1}
                ],
                "120_HT": [
                    {'a': [-1 * self.domain_size, -2 * self.domain_size, 0], 'b': [2.0, -1.0, 0], 'c': [0, 0, 1],
                     'stack_axis': 0},
                    {'a': [-1 * self.domain_size, 2 * self.domain_size, 0], 'b': [-2.0, -1.0, 0], 'c': [0, 0, 1],
                     'stack_axis': 0}
                ],
                "180": [
                    {'a': [1, 0, 0], 'b': [0, self.domain_size, 0], 'c': [0, 0, 1], 'stack_axis': 1},
                    {'a': [-1, 0, 0], 'b': [0, -self.domain_size, 0], 'c': [0, 0, 1], 'stack_axis': 1}
                ]
            },
        }
        return predefined_domain_wall_dict

    def get_domain_wall_dict(self):
        return {
            self.international_symbol: {
                str(self.domain_wall_tag): [
                    {'a': self.domainA_a, 'b': self.domainA_b, 'c': self.domainA_c, 'stack_axis': self.stack_axis},
                    {'a': self.domainB_a, 'b': self.domainB_b, 'c': self.domainB_c, 'stack_axis': self.stack_axis}
                ]
            }
        }

    def cut_and_rotate(self, atoms, a, b, c):
        slab = cut(atoms, a=a, b=b, c=c)
        rotate(slab, slab.cell[0], (0, 1, 0), slab.cell[1], (1, 0, 0))
        return slab

    def remove_close_atoms(self, atoms, cutoff=0.0):
        nl = NeighborList([cutoff / 2.0] * len(atoms), self_interaction=False, bothways=True)
        nl.update(atoms)
        indices_to_remove = set()
        for i in range(len(atoms)):
            indices, offsets = nl.get_neighbors(i)
            for idx in indices:
                if i < idx:
                    distance = atoms.get_distance(i, idx)
                    if distance < cutoff:
                        indices_to_remove.add(idx)
        atoms = atoms[[atom.index not in indices_to_remove for atom in atoms]]
        return atoms

    def calculate_lattice_strain(self, slab1, slab2):
        a1, b1, c1 = slab1.cell
        a2, b2, c2 = slab2.cell
        strain_a = (norm(a1) - norm(a2)) / norm(a2) * 100
        strain_b = (norm(b1) - norm(b2)) / norm(b2) * 100
        strain_c = (norm(c1) - norm(c2)) / norm(c2) * 100
        print("Lattice strain for DW:")
        print(f"strain along a (%): {strain_a:.2f}")
        print(f"strain along b (%): {strain_b:.2f}")
        print(f"strain along c (%): {strain_c:.2f}")

    def build_domain_wall(self):
        domain_wall_dict = self.get_domain_wall_dict()
        angles = domain_wall_dict[self.international_symbol][str(self.domain_wall_tag)]
        stack_axis = angles[0]['stack_axis']
        slabs = [self.cut_and_rotate(self.atoms, **{k: v for k, v in angle.items() if k in ('a', 'b', 'c')}) for angle
                 in angles]

        for i, slab in enumerate(slabs):
            slab = self.remove_close_atoms(slab, self.cutoff)

        stacked_slab = stack(*slabs, axis=stack_axis, maxstrain=None)
        stacked_slab = self.remove_close_atoms(stacked_slab, self.cutoff)
        
        return stacked_slab

class PolarizationEstimator:
    """
    This script uses the point charge model and you need to have both optimized CONTCAR and high symmtry structure POSCAR files to determine the polarization.
    """
    def __init__(self, centrosymmetric_structure, ferroelectric_structure, nominal_charges=None, cation_types=None, anion_types=None, ifnormalized=True):
        self.cs_struct = centrosymmetric_structure
        self.fe_struct = ferroelectric_structure
        self.cation_types = set(cation_types) if cation_types else set()
        self.anion_types = set(anion_types) if anion_types else set()
        self.nominal_charges = None
        self.ifnormalized = ifnormalized

        if nominal_charges is not None:
            self.nominal_charges = np.array(nominal_charges)
            if len(self.nominal_charges) != len(self.cs_struct):
                raise ValueError("Length of nominal charges must match the number of atoms.")
        elif self.cation_types and self.anion_types:
            self._assign_charges_based_on_types()
        else:
            raise ValueError("Either nominal_charges or both cation_types and anion_types must be specified.")

        if len(self.cs_struct) != len(self.fe_struct):
            raise ValueError("The number of atoms in both structures must be the same.")

    def _assign_charges_based_on_types(self):
        """
        Assign nominal charges based on cation and anion types.
        """
        symbols = self.cs_struct.get_chemical_symbols()
        charges = []
        for symbol in symbols:
            if symbol in self.cation_types:
                charges.append(1.0)  # Default cation charge
            elif symbol in self.anion_types:
                charges.append(-1.0)  # Default anion charge
            else:
                raise ValueError(f"Element '{symbol}' not found in cation_types or anion_types.")
        self.nominal_charges = np.array(charges)

    def compute_ionic_polarization(self):
        """
        Compute the ionic polarization direction vector (normalized).
        """
        pos_centro = self.cs_struct.get_positions(wrap=True)
        pos_ferro = self.fe_struct.get_positions(wrap=True)
        cell = self.cs_struct.get_cell()

        displacements = find_mic(pos_ferro - pos_centro, cell=cell)[0]
        dipole_contributions = self.nominal_charges[:, None] * displacements

        total_dipole = dipole_contributions.sum(axis=0)
        volume = self.cs_struct.get_volume()
        polarization_vector = total_dipole / volume
        if self.ifnormalized:
            return polarization_vector / np.linalg.norm(polarization_vector)
        else:
            return polarization_vector

class new_DomainWallSystem:
    def __init__(self, ferroelectric_structure, polarization_vector, domain_plane=(0, 0, 1), domain_angle=180, 
                 slab_size=1, stack_size=3, stack_axis=2, translation1=np.zeros(3), translation2=np.zeros(3), layer_distance=2):
        """
        Initialize the domain wall system with input parameters.
        """
        self.atoms = ferroelectric_structure
        self.P1 = polarization_vector
        self.domain_plane = np.array(domain_plane)
        self.domain_angle = np.radians(domain_angle)
        self.stack_axis = stack_axis
        self.translation1 = np.array(translation1)
        self.translation2 = np.array(translation2)
        self.layer_distance = layer_distance
        
        self.supercell_matrix = [slab_size] * 3
        self.supercell_matrix[self.stack_axis] = stack_size

    def find_minimal_orthogonal_vectors(self, c):
        """
        Find two minimal integer vectors orthogonal to the given vector.
        """
        def is_perpendicular(v1, v2):
            return np.dot(v1, v2) == 0
        def normalize_vector(v):
            g = reduce(gcd, v) if any(v) else 1
            return tuple(v // g)
        def vector_norm(v):
            return sum(abs(x) for x in v)
        c = np.array(c, dtype=int)
        candidates_a = []
        for x, y, z in product(range(-10, 11), repeat=3): 
            a = np.array([x, y, z])
            if np.all(a == 0): 
                continue
            if is_perpendicular(a, c):
                a = normalize_vector(a)
                candidates_a.append(a)
        candidates_a = sorted(set(candidates_a), key=vector_norm)
        a = candidates_a[0]
        candidates_b = []
        for x, y, z in product(range(-10, 11), repeat=3): 
            b = np.array([x, y, z])
            if np.all(b == 0) or np.array_equal(b, a):
                continue
            if is_perpendicular(b, c) and is_perpendicular(b, a):
                b = normalize_vector(b)
                candidates_b.append(b)
        candidates_b = sorted(set(candidates_b), key=vector_norm)
        b = candidates_b[0]
        return a, b

    def cut_and_rotate(self, atoms):
        """
        Cut and rotate the crystal to align with the specified domain plane.
        """
        a, b = self.find_minimal_orthogonal_vectors(self.domain_plane)
        slab = cut(atoms, a=a, b=b, c=self.domain_plane)
        rotate(slab, slab.cell[0], (1, 0, 0), slab.cell[1], (0, 1, 0))
        transformation_matrix_cut = np.array([a, b, self.domain_plane]).T
        P1_cut = np.linalg.inv(transformation_matrix_cut) @ self.P1
        new_basis = np.array([slab.cell[0], slab.cell[1], slab.cell[2]])
        target_basis = np.eye(3)
        rotation_matrix = np.linalg.inv(new_basis) @ target_basis
        self.P1 = rotation_matrix @ P1_cut
        return slab

    def calculate_rotation_matrix(self):
        """
        Calculate the rotation matrix for the specified domain angle.
        """
        P1_norm = self.P1 / np.linalg.norm(self.P1)
        if not np.isclose(P1_norm[0], 0):
            v = np.array([0, 1, 0]) 
        else:
            v = np.array([1, 0, 0])
        u = v - np.dot(v, P1_norm) * P1_norm
        u = u / np.linalg.norm(u)
        P2_norm = np.cos(self.domain_angle) * P1_norm + np.sin(self.domain_angle) * u
        dot_product = np.round(np.dot(P1_norm, P2_norm), 6)
        
        if np.isclose(dot_product, 1.0): 
            return np.eye(3) 
        elif np.isclose(dot_product, -1.0):
            if not np.isclose(P1_norm[0], 0):
                R = np.array([-P1_norm[1], P1_norm[0], 0])
            else:
                R = np.array([0, -P1_norm[2], P1_norm[1]])
        else:
            R = np.cross(P1_norm, P2_norm)
            
        R = R / np.linalg.norm(R)
        K = np.array([[0, -R[2], R[1]], [R[2], 0, -R[0]], [-R[1], R[0], 0]])
        I = np.eye(3)
        return I + np.sin(self.domain_angle) * K + (1 - np.cos(self.domain_angle)) * np.dot(K, K)

    def redifined_cell(self, atoms, new_cell):
        """
        Adjust the cell of an atom structure while preserving atomic positions.
        """
        atoms.set_cell(new_cell, scale_atoms=False)
        positions = atoms.get_positions()
        cell = atoms.get_cell()
        new_positions = np.dot(positions, np.linalg.inv(cell))
        atoms.set_positions(np.dot(new_positions, new_cell))
        atoms.center()
        return atoms

    def translate_atoms(self, atoms, translation):
        """
        Translate an Atoms object along a specified axis and apply periodic boundary conditions.
        """
        atoms.positions += translation
        atoms.wrap()
        return atoms

    def generate_domain_structure(self):
        """
        Generate a single-domain structure with a specified domain wall angle.
        """
        slab1 = self.cut_and_rotate(self.atoms.copy())
        slab1 = self.translate_atoms(slab1, self.translation1)
        slab1.wrap()
        slab1.center()
        
        slab2 = slab1.copy()
        rotation_matrix = self.calculate_rotation_matrix()
        slab2.positions = np.dot(slab2.positions, rotation_matrix.T)
        slab2.cell = np.dot(slab2.cell, rotation_matrix.T) 
        slab2 = self.redifined_cell(slab2, slab1.cell.copy())
        slab2 = self.translate_atoms(slab2, self.translation2)
        slab2.wrap()
        slab2.center()

        slab1 = slab1 * self.supercell_matrix
        slab2 = slab2 * self.supercell_matrix
        
        stacked_cell = slab1.cell.copy()
        stacked_cell[self.stack_axis, self.stack_axis] = slab1.cell[self.stack_axis, self.stack_axis] + slab2.cell[self.stack_axis, self.stack_axis] + self.layer_distance
        slab2.positions[:, self.stack_axis] += slab1.cell[self.stack_axis, self.stack_axis] + self.layer_distance
        stacked_positions = np.vstack([slab1.positions, slab2.positions])
        
        stacked_slab = Atoms(
            symbols=slab1.get_chemical_symbols() + slab2.get_chemical_symbols(),
            positions=stacked_positions,
            cell=stacked_cell,
            pbc=True
        )
        stacked_slab.wrap()
        stacked_slab.center()

        return stacked_slab

def _get_atoms_from_cif(mp_id):
    cif_str = get_cif_by_id(mp_id, primitive=False, origin=True)
    cif_file = io.StringIO(cif_str)
    atoms = read(cif_file, format='cif')
    return atoms

class DomainwallBuilder:
    def __init__(self, ferro_atoms, polarization_vector, **kwargs):
        self.ferro_atoms = ferro_atoms
        self.polarization_vector = polarization_vector
        self.kwargs = kwargs
        self.stacked_slab = None

    def _build(self):
        required_keys = [
            'domain_plane', 'domain_angle', 'slab_size',
            'stack_size', 'stack_axis', 'translation1',
            'translation2', 'layer_distance'
        ]

        # Check for missing required parameters
        for key in required_keys:
            if key not in self.kwargs:
                raise ValueError(f"Missing required parameter: '{key}'")

        self.stacked_slab = new_DomainWallSystem(
            ferroelectric_structure=self.ferro_atoms,
            polarization_vector=self.polarization_vector,
            domain_plane=self.kwargs['domain_plane'],
            domain_angle=self.kwargs['domain_angle'],
            slab_size=self.kwargs['slab_size'],
            stack_size=self.kwargs['stack_size'],
            stack_axis=self.kwargs['stack_axis'],
            translation1=self.kwargs['translation1'],
            translation2=self.kwargs['translation2'],
            layer_distance=self.kwargs['layer_distance']
        )
        return self.stacked_slab

class GTDomainwallBuilder(DomainwallBuilder):
    METADATA = {
        'ferro': 'qs-122740',
        'centro': 'qs-122739'
    }

    def __init__(self, **kwargs):
        centro_atoms = _get_atoms_from_cif(GTDomainwallBuilder.METADATA['centro'])
        ferro_atoms = _get_atoms_from_cif(GTDomainwallBuilder.METADATA['ferro'])
        polarization_vector = PolarizationEstimator(
            centrosymmetric_structure=centro_atoms,
            ferroelectric_structure=ferro_atoms,
            nominal_charges=kwargs.get('nominal_charges', None),
            cation_types=["Ge"],
            anion_types=["Te"],
            ifnormalized=kwargs.get('ifnormalized', True)
        ).compute_ionic_polarization()
        super().__init__(ferro_atoms, polarization_vector, **kwargs)
        self.centro_atoms = centro_atoms

    def build(self):
        return self._build()

In [4]:
from ase.io import read
from ase.visualize import view

refer_atoms = read("../examples/Domain_walls/GeTe/centro_GT.cif", format='cif') # qs-122739
base_atoms = read("../examples/Domain_walls/GeTe/ferro_GT.cif", format='cif') # qs-122740
polarization = PolarizationEstimator(refer_atoms, base_atoms, cation_types=["Ge"], anion_types=["Te"]).compute_ionic_polarization()
print(polarization)

stacked_slab = new_DomainWallSystem(base_atoms, polarization, domain_plane=(0, 0, 1), domain_angle=180, slab_size=1, stack_size=3, stack_axis=2, translation1=np.array([0, 0, 0]), translation2=np.array([0, 0, 0]), layer_distance=0).generate_domain_structure()
view(stacked_slab)

[0.5773393  0.57737879 0.57733272]


<Popen: returncode: None args: ['C:\\Users\\wangchangrui\\.conda\\envs\\Q-Ap...>

### ase_functions.py

In [ ]:
@cif_output
def new_domainwall_builder(**kwargs):
    if 'ferro_atoms' not in kwargs:
        raise ValueError("Missing required parameter: 'ferro_atoms' (CIF string)")
    try:
        cif_file = io.StringIO(kwargs['ferro_atoms'])
        kwargs['ferro_atoms'] = read(cif_file, format='cif')
    except Exception as e:
        raise ValueError(f"Invalid CIF data for 'ferro_atoms': {e}")
    return DomainwallBuilder(**kwargs)._build()

@cif_output
def gete_domainwall_builder(**kwargs):
    for prefix in ['domain_plane', 'translation1', 'translation2']:
    try:
        kwargs[prefix] = [kwargs.pop(f'{prefix}_{suffix}') for suffix in ['x', 'y', 'z']]
    except KeyError as e:
        raise ValueError(f"Missing required key: {e.args[0]} for prefix '{prefix}'. Ensure all components (x, y, z) are provided.")
    return GTDomainwallBuilder(**kwargs).build()

### modeling_functions.py

In [ ]:
@guide_register_func
def gt_domainwall(data):
    mode = data.get('mode')
    if mode == 'init':
        title = Description(name='GeTe 畴壁建模')
        domain_plane_x = SingleInput(name='畴壁平面指数 1', note='输入畴壁平面的 Miller 指数', id='domain_plane_x', 
                                     input_type='int', placeholder=0, default_value=0, is_required=1)
        domain_plane_y = SingleInput(name='畴壁平面指数 2', note='输入畴壁平面的 Miller 指数', id='domain_plane_y', 
                                     input_type='int', placeholder=0, default_value=0, is_required=1)
        domain_plane_z = SingleInput(name='畴壁平面指数 3', note='输入畴壁平面的 Miller 指数', id='domain_plane_z', 
                                     input_type='int', placeholder=1, default_value=1, is_required=1)
        domain_angle = SingleInput(name='畴壁角', note='畴壁夹角', id='domain_angle', input_type='int', 
                                   placeholder=180, default_value=180, min=0, max=180, is_required=1)
        slab_size = SingleInput(name='垂直于堆叠方向的重复次数', note='默认重复次数为 1', id='slab_size', 
                                input_type='int', placeholder=1, default_value=1, min=1, max=10, is_required=0)
        stack_size = SingleInput(name='堆叠方向的重复次数', note='默认重复次数为 3', id='stack_size', 
                                 input_type='int', placeholder=3, default_value=3, min=1, max=10, is_required=0)
        stack_axis = SingleFromList(name='堆叠轴', note='选择堆叠的方向', id='stack_axis', default_value=2, 
                                    list_value=[
                                        {'label': 'X', 'value': 0},
                                        {'label': 'Y', 'value': 1},
                                        {'label': 'Z', 'value': 2}
                                    ], 
                                    is_required=1)
        translation1_x = SingleInput(name='slab1 的平移矢量分量 1', note='单位：Å', id='translation1_x', 
                                     input_type='float', placeholder=0.0, default_value=0.0, is_required=0)
        translation1_y = SingleInput(name='slab1 的平移矢量分量 2', note='单位：Å', id='translation1_y', 
                                     input_type='float', placeholder=0.0, default_value=0.0, is_required=0)
        translation1_z = SingleInput(name='slab1 的平移矢量分量 3', note='单位：Å', id='translation1_z', 
                                     input_type='float', placeholder=0.0, default_value=0.0, is_required=0)
        translation2_x = SingleInput(name='slab2 的平移矢量分量 1', note='单位：Å', id='translation2_x', 
                                     input_type='float', placeholder=0.0, default_value=0.0, is_required=0)
        translation2_y = SingleInput(name='slab2 的平移矢量分量 2', note='单位：Å', id='translation2_y', 
                                     input_type='float', placeholder=0.0, default_value=0.0, is_required=0)
        translation2_z = SingleInput(name='slab2 的平移矢量分量 3', note='单位：Å', id='translation2_z', 
                                     input_type='float', placeholder=0.0, default_value=0.0, is_required=0)
        layer_distance = SingleInput(name='slab 间距', note='单位：Å', id='layer_distance',
                                     input_type='float', placeholder=2.0, default_value=2.0, min=0.0, max=10.0, is_required=0)
        
        return [title(), domain_plane_x(), domain_plane_y(), domain_plane_z(), domain_angle(), slab_size(), stack_size(), stack_axis(), 
                translation1_x(), translation1_y(), translation1_z(), translation2_x(), translation2_y(), translation2_z(), layer_distance()]

    elif mode == 'generate':
        value = data.get('value')
        cif_string = gete_domainwall_builder(**value)
        return {"file_content": cif_string, "file_format": "cif"}

In [ ]:
@guide_register_func
def Domain_wall(data):
    mode = data.get('mode')

    if mode == 'init':
        title = Description(name='title', note='基于输入结构的空间群和预设的极化方向构建铁电材料畴壁结构')
        atoms = StructureFromList(name='atoms', note='选择极化相基体结构，注意预设的极化方向为：R3c(1-11)、R3m(001)、P4mm(001)、Pmc21(user-defined)、customed(user-defined)',
                                  id='atoms', structure_type='crystal', output_format='cif', is_required=1)
        domain_wall_tag = SingleFromList(name='domain_wall_tag', note='根据提示选择畴壁类型或自定义畴壁类型customed',
                                         id='domain_wall_tag',
                                         list_value=[{'label': '适用于R 3 m、R 3 c空间群：71', 'value': '71'},
                                                     {'label': '适用于P 4 m m、P m c 21空间群：90', 'value': '90'},
                                                     {'label': '适用于R 3 m、R 3 c空间群：109', 'value': '109'},
                                                     {'label': '适用于P m c 21空间群：120_HH_TT', 'value': '120_HH_TT'},
                                                     {'label': '适用于P m c 21空间群：120_HT', 'value': '120_HT'},
                                                     {'label': '适用于R 3 m、R 3 c、P 4 m m、P m c 21空间群：180', 'value': '180'},
                                                     {'label': '适用于任意空间群：customed', 'value': 'customed'}],
                                         default_value='180',
                                         is_required=0)
        domainA_a_1 = SingleInput(name='domainA_a_1', note='domainA_a_1', id='domainA_a_1', input_type='int', min=-10, max=10, default_value= 1, is_required=0)
        domainA_a_2 = SingleInput(name='domainA_a_2', note='domainA_a_2', id='domainA_a_2', input_type='int', min=-10, max=10, default_value= 1, is_required=0)
        domainA_a_3 = SingleInput(name='domainA_a_3', note='domainA_a_3', id='domainA_a_3', input_type='int', min=-10, max=10, default_value= 0, is_required=0)
        domainA_b_1 = SingleInput(name='domainA_b_1', note='domainA_b_1', id='domainA_b_1', input_type='int', min=-10, max=10, default_value= 0, is_required=0)
        domainA_b_2 = SingleInput(name='domainA_b_2', note='domainA_b_2', id='domainA_b_2', input_type='int', min=-10, max=10, default_value= 0, is_required=0)
        domainA_b_3 = SingleInput(name='domainA_b_3', note='domainA_b_3', id='domainA_b_3', input_type='int', min=-10, max=10, default_value= 1, is_required=0)
        domainA_c_1 = SingleInput(name='domainA_c_1', note='domainA_c_1', id='domainA_c_1', input_type='int', min=-10, max=10, default_value= 1, is_required=0)
        domainA_c_2 = SingleInput(name='domainA_c_2', note='domainA_c_2', id='domainA_c_2', input_type='int', min=-10, max=10, default_value=-1, is_required=0)
        domainA_c_3 = SingleInput(name='domainA_c_3', note='domainA_c_3', id='domainA_c_3', input_type='int', min=-10, max=10, default_value= 0, is_required=0)
        domainB_a_1 = SingleInput(name='domainB_a_1', note='domainB_a_1', id='domainB_a_1', input_type='int', min=-10, max=10, default_value=-1, is_required=0)
        domainB_a_2 = SingleInput(name='domainB_a_2', note='domainB_a_2', id='domainB_a_2', input_type='int', min=-10, max=10, default_value=-1, is_required=0)
        domainB_a_3 = SingleInput(name='domainB_a_3', note='domainB_a_3', id='domainB_a_3', input_type='int', min=-10, max=10, default_value= 0, is_required=0)
        domainB_b_1 = SingleInput(name='domainB_b_1', note='domainB_b_1', id='domainB_b_1', input_type='int', min=-10, max=10, default_value= 0, is_required=0)
        domainB_b_2 = SingleInput(name='domainB_b_2', note='domainB_b_2', id='domainB_b_2', input_type='int', min=-10, max=10, default_value= 0, is_required=0)
        domainB_b_3 = SingleInput(name='domainB_b_3', note='domainB_b_3', id='domainB_b_3', input_type='int', min=-10, max=10, default_value=-1, is_required=0)
        domainB_c_1 = SingleInput(name='domainB_c_1', note='domainB_c_1', id='domainB_c_1', input_type='int', min=-10, max=10, default_value= 1, is_required=0)
        domainB_c_2 = SingleInput(name='domainB_c_2', note='domainB_c_2', id='domainB_c_2', input_type='int', min=-10, max=10, default_value=-1, is_required=0)
        domainB_c_3 = SingleInput(name='domainB_c_3', note='domainB_c_3', id='domainB_c_3', input_type='int', min=-10, max=10, default_value= 0, is_required=0)
        stack_axis = SingleInput(name='stack_axis', note='堆叠轴', id='stack_axis', input_type='int', min=0, max=2, default_value=0, is_required=0)
        domain_size = SingleInput(name='domain_size', note='畴壁尺寸', id='domain_size', input_type='float', min=1, max=10, default_value=3, is_required=0)
        cutoff = SingleInput(name='cutoff', note='根据截断删除重叠原子', id='cutoff', input_type='float', min=0, max=2, default_value=0, is_required=0)

        return [title(), atoms(), domain_wall_tag(), domainA_a_1(), domainA_a_2(), domainA_a_3(), domainA_b_1(), domainA_b_2(), domainA_b_3(), domainA_c_1(), domainA_c_2(), domainA_c_3(),
                domainB_a_1(), domainB_a_2(), domainB_a_3(), domainB_b_1(), domainB_b_2(), domainB_b_3(), domainB_c_1(), domainB_c_2(), domainB_c_3(), stack_axis(), domain_size(), cutoff()]

    elif mode == 'generate':
        value = data.get('value')
        cif_string = domain_wall(**value)
        return {'file_content': cif_string, 'file_format': 'cif'}

### 未定

In [5]:
import numpy as np

def calculate_vector_sum_difference(atoms):
    positions = atoms.get_positions()
    symbols = atoms.get_chemical_symbols()
    cation_positions = np.array([positions[i] for i, symbol in enumerate(symbols) if symbol in ["Ge"]])
    anion_positions = np.array([positions[i] for i, symbol in enumerate(symbols) if symbol in ["Te"]])
    cation_sum = np.sum(cation_positions, axis=0)
    anion_sum = np.sum(anion_positions, axis=0)
    return cation_sum - anion_sum

def calculate_angle(vector1, vector2):
    v1 = np.array(vector1)
    v2 = np.array(vector2)
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        raise ValueError("One or both of the vectors have zero magnitude.")
    cos_theta = dot_product / (magnitude_v1 * magnitude_v2)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    angle_radians = np.arccos(cos_theta)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

vector1 = calculate_vector_sum_difference(slab1)
vector2 = calculate_vector_sum_difference(slab2)

angle = calculate_angle(vector1, vector2)
print(f"The angle between vector1 and vector2 is {angle:.2f} degrees.")

The angle between vector1 and vector2 is 180.00 degrees.
